In [ ]:
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

In [ ]:
t = torch.cuda.get_device_properties(0).total_memory
print("GPU Memory: {}".format(round(t / 1e9, 2)))

GPU Memory: 15.81


In [ ]:
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15079MB, multi_processor_count=40)

In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 204kB 5.3MB/s 
     |████████████████████████████████| 1.4MB 19.6MB/s 
     |████████████████████████████████| 71kB 9.5MB/s 
     |████████████████████████████████| 317kB 25.6MB/s 
     |████████████████████████████████| 778kB 39.0MB/s 
     |████████████████████████████████| 3.0MB 51.4MB/s 
     |████████████████████████████████| 102kB 14.8MB/s 
     |████████████████████████████████| 112kB 48.5MB/s 
     |████████████████████████████████| 102kB 13.9MB/s 
     |████████████████████████████████| 163kB 51.1MB/s 
     |████████████████████████████████| 890kB 52.6MB/s 
     |████████████████████████████████| 1.1MB 51.2MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
     |████████████████████████████████| 71kB 11.7MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=d446062ec4bfb29e0c7e2e0664e7e574656d78442794d57926ba3eff0cf37c37
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a8258

In [ ]:
# !pip install transformers==2.10.0

In [ ]:
# !pip install simpletransformers==0.26.0

In [ ]:
from simpletransformers.ner import NERModel

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
le = LabelEncoder()

def enc(sid):
  sid = [str(id_) for id_ in sid]
  return le.fit_transform(sid)

def drop_codes(df):
  return df.drop(['codes'], axis=1), df.codes

In [ ]:
train = pd.read_csv('drive/My Drive/data-ner/train_ner.csv', header=0)
# train = pd.read_csv('drive/My Drive/data-ner/train_ner_translated.csv', header=0)
train.sentence_id = enc(train.sentence_id)
train, train_codes = drop_codes(train)
print(train.shape)
train.head(5)

(1498953, 4)


,sentence_id,words,labels,country
0,0,APRESENTAÇÃO,O,B
1,0,O,B,B
2,0,Brasil,I,B
3,0,é,I,B
4,0,um,I,B


In [ ]:
val = pd.read_csv('drive/My Drive/data-ner/val_ner.csv', header=0)
val.sentence_id = enc(val.sentence_id)
val, val_codes = drop_codes(val)
print(val.shape)
val.head(5)

(318402, 4)


,sentence_id,words,labels,country
0,0,ELEZIONI,O,S
1,0,2018,O,S
2,0,·,O,S
3,0,PROGRAMMA,O,S
4,0,DI,O,S


In [ ]:
test = pd.read_csv('drive/My Drive/data-ner/test_ner.csv', header=0)
test.sentence_id = enc(test.sentence_id)
test, test_codes = drop_codes(test)
print(test.shape)
test.head(5)

(318995, 4)


,sentence_id,words,labels,country
0,0,Projeto,O,B
1,0,Brasil,O,B
2,0,Novo,O,B
3,0,Plataforma,O,B
4,0,de,O,B


In [ ]:
train.words = train.words.apply(lambda x: str(x))
test.words = test.words.apply(lambda x: str(x))
val.words = val.words.apply(lambda x: str(x))

In [ ]:
model = NERModel(#'bert',
                 #'drive/My Drive/bert-lm/',
                 #'bert-base-multilingual-cased',
                 'xlmroberta',
                 'drive/My Drive/roberta-lm/',
                #  'xlm-roberta-base',
                 #'drive/My Drive/roberta-lm-trans-comb/',
                 labels=['B', 'I', 'O'],
                 args={'train_batch_size': 32, 
                        'eval_batch_size': 32,
                        'num_train_epochs': 2,
                        'max_seq_length': 200,
                        'save_steps': 0,
                        'evaluate_during_training': True,
                        'evaluate_during_training_steps': int(train.shape[0] / 32),
                        'evaluate_during_training_verbose': True,
                        'fp16': False,
                        'overwrite_output_dir': True,
                        'reprocess_input_data': True,
                        'learning_rate': 2e-5,
                        'manual_seed':42
                        }
                 )

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-st

In [ ]:
# model.train_model(train_data=train, eval_data=val)

KeyboardInterrupt: ignored

In [ ]:
def f_macro(true, preds):
  return metrics.f1_score(true, preds, average='macro')

def f_micro(true, preds):
  return metrics.f1_score(true, preds, average='micro')

def softmax(logits):
  return np.exp(logits) / np.sum(np.exp(logits), axis=0)

In [ ]:
result, model_outputs, preds_list = model.eval_model(test, verbose=True)

In [ ]:
result

In [ ]:
preds = [item for sublist in preds_list for item in sublist]

In [ ]:
def pad_preds(sentence, preds):
  print("{} {} -> {}".format(len(sentence), len(preds), len(preds + [preds[-1]] * (len(sentence) - len(preds)))))
  return preds + [preds[-1]] * (len(sentence) - len(preds))

In [ ]:
len(preds_list)

In [ ]:
test_to_preds = {"sid": [], "words": [], "true": [], "preds": []}
for i, sid in test.groupby(by='sentence_id'):
  if sid.shape[0] != len(preds_list[i]):
    preds_list[i] += preds_list[i][-1] * (sid.shape[0] - len(preds_list[i]))
  test_to_preds["sid"].extend(list(sid.sentence_id.values))
  test_to_preds["words"].extend(list(sid.words.values))
  test_to_preds["true"].extend(list(sid.labels.values))
  test_to_preds["preds"].extend(preds_list[i])

In [ ]:
[len(item) for item in test_to_preds.values()]

In [ ]:
output_df = pd.DataFrame(test_to_preds)

In [ ]:
output_df["codes"] = test_codes

In [ ]:
output_df["country"] = test.country

In [ ]:
output_df.head()

In [ ]:
output_df.to_csv('outputs.csv', index=False)

In [ ]:
output_df.preds.value_counts()

In [ ]:
f_macro(output_df.true, output_df.preds)